## bark测试
- 中文读的一股老外味

In [ ]:
from transformers import pipeline
from pydub import AudioSegment
import numpy as np
pipe = pipeline("text-to-speech", model="suno/bark-small")

/home/haitang/miniconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
text = "这是一个测试,主要是为了测试音频数据生成的效果怎么样"
output = pipe(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [5]:
from IPython.display import Audio
Audio(output["audio"], rate=output["sampling_rate"])

In [8]:
# 将numpy数组转换为AudioSegment
audio_array = output["audio"]
sampling_rate = output["sampling_rate"]

# 需要将音频数据转换为16位PCM格式（MP3的标准输入格式）
audio_array = (audio_array * 32767).astype(np.int16)  # 转换为16位PCM
if audio_array.ndim == 2:  # 如果是立体声
    audio_segment = AudioSegment(
        audio_array.tobytes(), 
        frame_rate=sampling_rate,
        sample_width=2,  # 16位=2字节
        channels=2
    )
else:  # 单声道
    audio_segment = AudioSegment(
        audio_array.tobytes(),
        frame_rate=sampling_rate,
        sample_width=2,
        channels=1
    )

# 导出为MP3
audio_segment.export("output.mp3", format="mp3", bitrate="64k")

<_io.BufferedRandom name='output.mp3'>

In [1]:
import os
os.environ['PYTHONPATH']='/home/haitang/Downloads/CosyVoice/third_party/Matcha-TTS'
os.chdir('/home/haitang/Downloads/CosyVoice')
import sys
sys.path.append('/home/haitang/Downloads/CosyVoice')

## cosyvoice测试
- 还挺好用的

In [2]:
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio

/home/haitang/miniconda3/envs/minimind/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


failed to import ttsfrd, use wetext instead


In [ ]:

## cosyvoice2 usage
cosyvoice2 = CosyVoice('pretrained_models/CosyVoice-300M-SFT', load_jit=False, load_onnx=False, load_trt=False)
# sft usage
prompt_speech_16k = load_wav('zero_shot_prompt.wav', 16000)
for i, j in enumerate(cosyvoice2.inference_zero_shot('收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', '希望你以后能够做的比我还好呦。', prompt_speech_16k, stream=True)):
    torchaudio.save('zero_shot_{}.wav'.format(i), j['tts_speech'], cosyvoice2.sample_rate)

## cosyvoice usage
cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-SFT', load_jit=True, load_onnx=False, fp16=True)
# sft usage
print(cosyvoice.list_avaliable_spks())
# change stream=True for chunk stream inference
for i, j in enumerate(cosyvoice.inference_sft('你好，我是通义生成式语音大模型，请问有什么可以帮您的吗？', '中文女', stream=False)):
    torchaudio.save('sft_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-25Hz') # or change to pretrained_models/CosyVoice-300M for 50Hz inference
# zero_shot usage, <|zh|><|en|><|jp|><|yue|><|ko|> for Chinese/English/Japanese/Cantonese/Korean
prompt_speech_16k = load_wav('zero_shot_prompt.wav', 16000)
for i, j in enumerate(cosyvoice.inference_zero_shot('收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', '希望你以后能够做的比我还好呦。', prompt_speech_16k, stream=False)):
    torchaudio.save('zero_shot_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)
# cross_lingual usage
prompt_speech_16k = load_wav('cross_lingual_prompt.wav', 16000)
for i, j in enumerate(cosyvoice.inference_cross_lingual('<|en|>And then later on, fully acquiring that company. So keeping management in line, interest in line with the asset that\'s coming into the family is a reason why sometimes we don\'t buy the whole thing.', prompt_speech_16k, stream=False)):
    torchaudio.save('cross_lingual_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)
# vc usage
prompt_speech_16k = load_wav('zero_shot_prompt.wav', 16000)
source_speech_16k = load_wav('cross_lingual_prompt.wav', 16000)
for i, j in enumerate(cosyvoice.inference_vc(source_speech_16k, prompt_speech_16k, stream=False)):
    torchaudio.save('vc_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-Instruct')
# instruct usage, support <laughter></laughter><strong></strong>[laughter][breath]
for i, j in enumerate(cosyvoice.inference_instruct('在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。', '中文男', 'Theo \'Crimson\', is a fiery, passionate rebel leader. Fights with fervor for justice, but struggles with impulsiveness.', stream=False)):
    torchaudio.save('instruct_{}.wav'.format(i), j['tts_speech'], cosyvoice.sample_rate)


## clap测试

In [4]:
from msclap import CLAP

In [19]:
clap_model = CLAP(version='clapcap',use_cuda=True)

2025-07-24 11:33:15,174 DEBUG https://huggingface.co:443 "HEAD /microsoft/msclap/resolve/main/clapcap_weights_2023.pth HTTP/1.1" 302 0
2025-07-24 11:33:16,313 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/config.json HTTP/1.1" 200 0
2025-07-24 11:33:16,772 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/config.json HTTP/1.1" 200 0
2025-07-24 11:33:17,106 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/config.json HTTP/1.1" 200 0
2025-07-24 11:33:17,594 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-24 11:33:17,934 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-07-24 11:33:19,998 DEBUG https://huggingface.co:443 "HEAD /gpt2/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-07-24 11:33:20,337 DEBUG https://huggingface.co:443 "GET /api/models/gpt2/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 307 133
2025-0

In [21]:
audio_embeddings = clap_model.generate_caption(["/home/haitang/Downloads/CosyVoice/zero_shot_0.wav"])

In [23]:
audio_embeddings

['A man is repeating a message over and over. ']

## wav2vec测试

In [ ]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np

# load pretrained model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# load audio
audio_path= "/home/haitang/Downloads/CosyVoice/zero_shot_0.wav"
audio_input, sample_rate = sf.read(audio_path)
# 将sample_rate转换为16000
if sample_rate != 16000:
    audio_input = audio_input.astype(np.float32)  # 转为float32
    audio_input = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(
        torch.tensor(audio_input).unsqueeze(0)
    ).squeeze(0).numpy()
    sample_rate = 16000

# pad input values and return pt tensor
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# INFERENCE

# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])

2025-07-24 13:53:00,071 DEBUG https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/preprocessor_config.json HTTP/1.1" 307 0
2025-07-24 13:53:00,178 DEBUG https://huggingface.co:443 "HEAD /api/resolve-cache/models/facebook/wav2vec2-base-960h/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/preprocessor_config.json HTTP/1.1" 200 0
2025-07-24 13:53:00,501 DEBUG https://huggingface.co:443 "HEAD /facebook/wav2vec2-base-960h/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
2025-07-24 13:53:00,606 DEBUG https://huggingface.co:443 "HEAD /api/resolve-cache/models/facebook/wav2vec2-base-960h/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/tokenizer_config.json HTTP/1.1" 200 0
2025-07-24 13:53:00,938 DEBUG https://huggingface.co:443 "GET /api/models/facebook/wav2vec2-base-960h/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
2025-07-24 13:53:01,274 DEBUG https://huggingface.co:443 "GET /api/models/facebook/wav2vec2-base-960h/tree/main/additional_chat_temp

In [39]:
## 播放audio_input
from IPython.display import Audio
Audio(audio_input, rate=sample_rate)

- 好像仅支持英文？？试试别的wav2vec

In [ ]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

2025-07-24 14:14:26,302 DEBUG https://huggingface.co:443 "HEAD /jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/resolve/main/preprocessor_config.json HTTP/1.1" 307 0
2025-07-24 14:14:26,413 DEBUG https://huggingface.co:443 "HEAD /api/resolve-cache/models/jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/99ccb2737be22b8bb50dcfcc39ad4d567fb90cfd/preprocessor_config.json HTTP/1.1" 200 0
2025-07-24 14:14:26,743 DEBUG https://huggingface.co:443 "HEAD /jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/resolve/main/tokenizer_config.json HTTP/1.1" 404 0
2025-07-24 14:14:27,072 DEBUG https://huggingface.co:443 "HEAD /jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/resolve/main/config.json HTTP/1.1" 307 0
2025-07-24 14:14:27,183 DEBUG https://huggingface.co:443 "HEAD /api/resolve-cache/models/jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/99ccb2737be22b8bb50dcfcc39ad4d567fb90cfd/config.json HTTP/1.1" 200 0
2025-07-24 14:14:27,507 DEBUG https://huggingface.co:443 "HEAD /jonatasgr

2025-07-24 14:14:33,120 DEBUG https://huggingface.co:443 "HEAD /jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/resolve/refs%2Fpr%2F5/model.safetensors.index.json HTTP/1.1" 404 0
2025-07-24 14:14:33,439 DEBUG https://huggingface.co:443 "HEAD /jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn/resolve/refs%2Fpr%2F5/model.safetensors HTTP/1.1" 302 0


In [52]:
# load audio
audio_path= "/home/haitang/Downloads/CosyVoice/zero_shot_0.wav"
audio_input, sample_rate = sf.read(audio_path)

# 将音频速度翻倍但不改变音调
audio_input = librosa.effects.time_stretch(audio_input, rate=2.0)

# 将sample_rate转换为16000
if sample_rate != 16000:
    audio_input = audio_input.astype(np.float32)  # 转为float32
    audio_input = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(
        torch.tensor(audio_input).unsqueeze(0)
    ).squeeze(0).numpy()
    sample_rate = 16000
    

# pad input values and return pt tensor
input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

# INFERENCE

# retrieve logits & take argmax
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe
transcription = processor.decode(predicted_ids[0])

2025-07-24 14:16:15,408 DEBUG bytecode dump:
>          0	NOP(arg=None, lineno=1141)
           2	LOAD_FAST(arg=0, lineno=1144)
           4	LOAD_CONST(arg=1, lineno=1144)
           6	BINARY_SUBSCR(arg=None, lineno=1144)
           8	STORE_FAST(arg=3, lineno=1144)
          10	LOAD_FAST(arg=1, lineno=1145)
          12	UNARY_NEGATIVE(arg=None, lineno=1145)
          14	LOAD_FAST(arg=3, lineno=1145)
          16	DUP_TOP(arg=None, lineno=1145)
          18	ROT_THREE(arg=None, lineno=1145)
          20	COMPARE_OP(arg=1, lineno=1145)
          22	POP_JUMP_IF_FALSE(arg=17, lineno=1145)
          24	LOAD_FAST(arg=1, lineno=1145)
          26	COMPARE_OP(arg=1, lineno=1145)
          28	POP_JUMP_IF_FALSE(arg=21, lineno=1145)
          30	JUMP_FORWARD(arg=2, lineno=1145)
>         32	POP_TOP(arg=None, lineno=1145)
          34	JUMP_FORWARD(arg=2, lineno=1145)
>         36	LOAD_CONST(arg=1, lineno=1146)
          38	STORE_FAST(arg=3, lineno=1146)
>         40	LOAD_FAST(arg=0, lineno=1148)
     

In [53]:
transcription

'就 在 好 友瑶 总远 风济 来 的争着礼物 纳 份 意外 的经 行 与申 真 的祝 服 让我 金 忠 中充 男 了 田 蜜 的 坏 的  叫 荣 如 花人班 战 放'

In [54]:
Audio(audio_input, rate=sample_rate)

- 效果还不错，就是占用token有点多。如果0.5s一个chunk就好了

## Paraformer

In [57]:
from funasr import AutoModel
# paraformer-zh is a multi-functional asr model
# use vad, punc, spk or not as you need
model = AutoModel(model="paraformer-zh", model_revision="v2.0.4",
                  vad_model="fsmn-vad", vad_model_revision="v2.0.4",
                  punc_model="ct-punc-c", punc_model_revision="v2.0.4",
                  # spk_model="cam++", spk_model_revision="v2.0.2",
                  )
res = model.generate(input=audio_path, 
                     batch_size_s=300, 
                     hotword='魔搭')
print(res)

2025-07-24 14:31:47,612 DEBUG Starting new HTTPS connection (1): pypi.org:443


funasr version: 1.2.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel


2025-07-24 14:31:48,034 DEBUG https://pypi.org:443 "GET /pypi/funasr/json HTTP/1.1" 200 37784
2025-07-24 14:31:48,104 INFO download models from model hub: ms
2025-07-24 14:31:48,115 DEBUG Starting new HTTPS connection (1): www.modelscope.cn:443


You are using the latest version of funasr-1.2.6


2025-07-24 14:31:48,343 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/speech_seaco_paraformer_large_asr_nat-zh-cn-16k-common-vocab8404-pytorch/revisions HTTP/1.1" 200 676
2025-07-24 14:31:48,346 - modelscope - INFO - Use user-specified model revision: v2.0.4
2025-07-24 14:31:48,710 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/speech_seaco_paraformer_large_asr_nat-zh-cn-16k-common-vocab8404-pytorch/repo/files?Revision=v2.0.4&Recursive=True HTTP/1.1" 200 None
2025-07-24 14:31:51,263 INFO Loading pretrained params from /home/haitang/.cache/modelscope/hub/iic/speech_seaco_paraformer_large_asr_nat-zh-cn-16k-common-vocab8404-pytorch/model.pt
2025-07-24 14:31:51,276 INFO ckpt: /home/haitang/.cache/modelscope/hub/iic/speech_seaco_paraformer_large_asr_nat-zh-cn-16k-common-vocab8404-pytorch/model.pt
2025-07-24 14:31:52,417 INFO scope_map: ['module.', 'None']
2025-07-24 14:31:52,418 INFO excludes: None
2025-07-24 14:31:52,562 INFO Loading ckpt: /home/haitang/.cache/mo

2025-07-24 14:31:53,540 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/speech_fsmn_vad_zh-cn-16k-common-pytorch/revisions HTTP/1.1" 200 1016
2025-07-24 14:31:53,544 - modelscope - INFO - Use user-specified model revision: v2.0.4
2025-07-24 14:31:53,768 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/speech_fsmn_vad_zh-cn-16k-common-pytorch/repo/files?Revision=v2.0.4&Recursive=True HTTP/1.1" 200 None
2025-07-24 14:31:53,825 INFO Loading pretrained params from /home/haitang/.cache/modelscope/hub/iic/speech_fsmn_vad_zh-cn-16k-common-pytorch/model.pt
2025-07-24 14:31:53,827 INFO ckpt: /home/haitang/.cache/modelscope/hub/iic/speech_fsmn_vad_zh-cn-16k-common-pytorch/model.pt
2025-07-24 14:31:53,850 INFO scope_map: ['module.', 'None']
2025-07-24 14:31:53,851 INFO excludes: None
2025-07-24 14:31:53,855 INFO Loading ckpt: /home/haitang/.cache/modelscope/hub/iic/speech_fsmn_vad_zh-cn-16k-common-pytorch/model.pt, status: <All keys matched successfully>
2025-07-24 14:31:53

2025-07-24 14:31:54,077 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/punc_ct-transformer_zh-cn-common-vocab272727-pytorch/revisions HTTP/1.1" 200 735
2025-07-24 14:31:54,080 - modelscope - INFO - Use user-specified model revision: v2.0.4
2025-07-24 14:31:54,320 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/punc_ct-transformer_zh-cn-common-vocab272727-pytorch/repo/files?Revision=v2.0.4&Recursive=True HTTP/1.1" 200 None
2025-07-24 14:31:55,275 INFO Loading pretrained params from /home/haitang/.cache/modelscope/hub/iic/punc_ct-transformer_zh-cn-common-vocab272727-pytorch/model.pt
2025-07-24 14:31:55,278 INFO ckpt: /home/haitang/.cache/modelscope/hub/iic/punc_ct-transformer_zh-cn-common-vocab272727-pytorch/model.pt
2025-07-24 14:31:55,514 INFO scope_map: ['module.', 'None']
2025-07-24 14:31:55,516 INFO excludes: None
2025-07-24 14:31:55,546 INFO Loading ckpt: /home/haitang/.cache/modelscope/hub/iic/punc_ct-transformer_zh-cn-common-vocab272727-pytorch/model.pt, 

[{'key': 'zero_shot_0', 'text': '收到好友，从远方寄来的生日礼物，那份意外的惊喜与深深的祝福，让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', 'timestamp': [[350, 590], [590, 710], [710, 930], [930, 1170], [1390, 1610], [1610, 1830], [1830, 2070], [2090, 2250], [2250, 2370], [2370, 2510], [2510, 2670], [2670, 2870], [2870, 3010], [3010, 3250], [3890, 4090], [4090, 4330], [4390, 4510], [4510, 4750], [4750, 4890], [4890, 5050], [5050, 5290], [5410, 5650], [5670, 5830], [5830, 6050], [6050, 6190], [6190, 6350], [6350, 6590], [6950, 7130], [7130, 7270], [7270, 7490], [7490, 7730], [7910, 8150], [8150, 8330], [8330, 8530], [8530, 8690], [8690, 8850], [8850, 9030], [9030, 9270], [9270, 9510], [10090, 10330], [10350, 10590], [10730, 10950], [10950, 11130], [11130, 11310], [11310, 11550], [11570, 11790], [11790, 12145]]}]


In [60]:
#查看模型结构
model.model

SeacoParaformer(
  (specaug): SpecAugLFR(
    (freq_mask): MaskAlongAxisLFR(mask_width_range=[0, 30], num_mask=1, axis=freq)
    (time_mask): MaskAlongAxisLFR(mask_width_range=[0, 12], num_mask=1, axis=time)
  )
  (encoder): SANMEncoder(
    (embed): SinusoidalPositionEncoder()
    (encoders0): MultiSequential(
      (0): EncoderLayerSANM(
        (self_attn): MultiHeadedAttentionSANM(
          (linear_out): Linear(in_features=512, out_features=512, bias=True)
          (linear_q_k_v): Linear(in_features=560, out_features=1536, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (fsmn_block): Conv1d(512, 512, kernel_size=(11,), stride=(1,), groups=512, bias=False)
          (pad_fn): ConstantPad1d(padding=(5, 5), value=0.0)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=